In [1]:
import requests

In [7]:
class ReadTokens:
    def __init__(self, api, token_id, token_ls_url='https://api.ergo.aap.cornell.edu/api/v1/tokens/'):
        self.api = api
        self.token_id = token_id
        self.token_ls = token_ls_url

    def get_api_data(self, api_url):
        try:
            # Send a GET request to the API
            response = requests.get(api_url)
    
            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Parse the response as JSON (assuming the API returns JSON data)
                data = response.json()
                return data
            else:
                print(f"Failed to retrieve data: Status code {response.status_code}")
                return None
    
        except requests.exceptions.RequestException as e:
            # Handle any exceptions that occur during the request
            print(f"An error occurred: {e}")
            return None

    def find_token_in_wallet(self, wallet):
        url = '{}/{}'.format(self.api, wallet)
        wallet_data = self.get_api_data(url)

        wallet_contents = wallet_data['items']
        for contents in wallet_contents:
            if contents['assets']:
                for items in contents['assets']:
                    token_id = items['tokenId']
                    print(token_id)
                    if token_id == self.token_id:
                        return True
        return False
                        
    def get_token_description(self):
        url = '{}/{}'.format(self.token_ls, self.token_id)
        data = self.get_api_data(url)

        token_description = data['description']
        return token_description, data

In [10]:
url = 'https://api.ergo.aap.cornell.edu/api/v1/boxes/byAddress/'
token_id = 'd36cf1d772f6fa636e28e7db70aaa6a7f9baf07e4e0c1ccd79ad6dd1f625c47c'
wallet = '9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn'
reader = ReadTokens(url, token_id)


In [11]:
in_wallet = reader.find_token_in_wallet(wallet)
in_wallet

01dce8a5632d19799950ff90bca3b5d0ca3ebfa8aaafd06f0cc6dd1e97150e7f
6ed546c6c771ec276bfc3bfbf44dff5884d7a9fc546c0fdfd029a08a281e9a3e
fa76ac3f7b1511bb0ed25d58af6c849545c3bd8f73c08cb6ac778755b16d3a65
01dce8a5632d19799950ff90bca3b5d0ca3ebfa8aaafd06f0cc6dd1e97150e7f
6ed546c6c771ec276bfc3bfbf44dff5884d7a9fc546c0fdfd029a08a281e9a3e
fa76ac3f7b1511bb0ed25d58af6c849545c3bd8f73c08cb6ac778755b16d3a65
6ed546c6c771ec276bfc3bfbf44dff5884d7a9fc546c0fdfd029a08a281e9a3e
fa76ac3f7b1511bb0ed25d58af6c849545c3bd8f73c08cb6ac778755b16d3a65
6ed546c6c771ec276bfc3bfbf44dff5884d7a9fc546c0fdfd029a08a281e9a3e
fa76ac3f7b1511bb0ed25d58af6c849545c3bd8f73c08cb6ac778755b16d3a65
6ed546c6c771ec276bfc3bfbf44dff5884d7a9fc546c0fdfd029a08a281e9a3e
fa76ac3f7b1511bb0ed25d58af6c849545c3bd8f73c08cb6ac778755b16d3a65
01dce8a5632d19799950ff90bca3b5d0ca3ebfa8aaafd06f0cc6dd1e97150e7f
6ed546c6c771ec276bfc3bfbf44dff5884d7a9fc546c0fdfd029a08a281e9a3e
fa76ac3f7b1511bb0ed25d58af6c849545c3bd8f73c08cb6ac778755b16d3a65
b7912b3d75e8faa02fdf17a92

False

In [4]:
token_description, data = reader.get_token_description()

In [5]:
import json


dictionary = json.loads(token_description)

print(dictionary)

{'address': '9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn', 'erg': 25, 'rsn': 75, 'minimumPayout': 10, 'description': 'This is a test token minted with Fleet SDK for the sigmanaut mining pool'}


In [6]:
data

{'id': 'd36cf1d772f6fa636e28e7db70aaa6a7f9baf07e4e0c1ccd79ad6dd1f625c47c',
 'boxId': 'c173e963f7f4abced195603128dc032e3a9879545167001add7d7edc44c8f5ae',
 'emissionAmount': 1,
 'name': 'Sigmanaut Mining Pool Configuration NFT - Season 0',
 'description': '{"address":"9eg7v2nkypUZbdyvSKSD9kg8FNwrEdTrfC2xdXWXmEpDAFEtYEn","erg":25,"rsn":75,"minimumPayout":10,"description":"This is a test token minted with Fleet SDK for the sigmanaut mining pool"}',
 'type': 'EIP-004',
 'decimals': 0}

In [20]:
import pandas as pd
import re

with open('../Mining-Reward-Tokens/README.md', 'r') as f:
    content = f.read()

table_pattern = re.compile(r'\|(.+\|)+\n\|(-+\|)+\n(\|(.+\|)+\n)*')
match = table_pattern.search(content)

if match:
    table_str = match.group()
else:
    raise ValueError("No table found in the README file.")

# Step 3: Convert the table to a DataFrame
# Split the table string into lines and then split each line into columns
lines = table_str.strip().split('\n')
columns = lines[0].strip().split('|')[1:-1]  # Extract column names
data = [line.strip().split('|')[1:-1] for line in lines[2:]]  # Extract rows

# Create a DataFrame
df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
print(df)

ValueError: No table found in the README file.

In [16]:
table_data

"# Mining-Reward-Tokens\nSigmanaut Mining Pool Mining Reward Tokens\n\n\nTo add a token to the list, simply create a PR that adds your token to the list. A token name should not contain whitespace, so if your token does have whitespace in the name make sure to replace/remove it. To avoid clutter in the token list add NFT's to the separate list below.\n\n## Supported token list\n| Token ID | Token Name | Token decimals |\n| --- | --- | --- |\n| 03faf2cb329f2e90d6d23b58d91bbb6c046aa143261cc21f52fbe2824bfcbf04 | SigUSD | 2 |\n| 8b08cdd5449a9592a9e79711d7d79249d7a03c535d17efaee83e216e80a44c4b | RSN | 3 |\n| 01dce8a5632d19799950ff90bca3b5d0ca3ebfa8aaafd06f0cc6dd1e97150e7f | CYPX | 4 |\n| b0b312cde931c8bbdac0dac5bfd8e2c03bf4611275dc967988c8d15bd5ec20e0 | Bober | 3 |\n| 956bcab1ea3ed40c91bf845fe4f2dee727fe130a95f244a41a6ee7a25f14e23f | Gluon | 6 |\n| 0cd8c9f416e5b1ca9f986a7f10a84191dfb85941619e49e53c0dc30ebf83324b | COMET | 0 |  \n| 7a51950e5f548549ec1aa63ffdc38279505b11e7e803d01bcf8347e0123c

In [50]:
import re

def extract_supported_tokens_table(readme_content):
    # Define the pattern for the Supported Tokens table
    pattern = r"(?s)Supported token list(.+?)###"
    
    # Search for the pattern in the README content
    match = re.search(pattern, readme_content)
    if match:
        # Extract the table content
        table_content = match.group(1).strip()
        return table_content
    else:
        return None

def parse_table(table_content):
    # Split the content into lines
    lines = table_content.split('\n')
    
    # Extract the headers
    headers = lines[0].strip('|').split('|')
    headers = [header.strip() for header in headers]
    
    # Extract the rows
    rows = []
    for line in lines[2:]:
        row = line.strip('|').split('|')
        row = [cell.strip() for cell in row]
        if len(headers) == len(row):
            rows.append(row)
    
    return headers, rows

def main():
    # Read the README file
    with open('../Mining-Reward-Tokens/README.md', 'r') as file:
        readme_content = file.read()
    
    # Extract the Supported Tokens table
    table_content = extract_supported_tokens_table(readme_content)
    print(table_content)
    
    if table_content:
        headers, rows = parse_table(table_content)
        
        # Print the headers and rows
        print("Headers:", headers)
        for row in rows:
            print("Row:", row)
        df = pd.DataFrame(rows, columns=headers)
        df.to_csv('../Mining-Reward-Tokens/supported-tokens.csv')
        return df
    else:
        print("Supported Tokens table not found in the README.")
        

if __name__ == "__main__":
    df = main()


| Token ID | Token Name | Token decimals |
| --- | --- | --- |
| 1 | ERGO | 0 |
| 03faf2cb329f2e90d6d23b58d91bbb6c046aa143261cc21f52fbe2824bfcbf04 | SigUSD | 2 |
| 8b08cdd5449a9592a9e79711d7d79249d7a03c535d17efaee83e216e80a44c4b | RSN | 3 |
| 01dce8a5632d19799950ff90bca3b5d0ca3ebfa8aaafd06f0cc6dd1e97150e7f | CYPX | 4 |
| b0b312cde931c8bbdac0dac5bfd8e2c03bf4611275dc967988c8d15bd5ec20e0 | Bober | 3 |
| 956bcab1ea3ed40c91bf845fe4f2dee727fe130a95f244a41a6ee7a25f14e23f | Gluon | 6 |
| 0cd8c9f416e5b1ca9f986a7f10a84191dfb85941619e49e53c0dc30ebf83324b | COMET | 0 |
| 7a51950e5f548549ec1aa63ffdc38279505b11e7e803d01bcf8347e0123c88b0 | rsBTC | 8 |

## Supported NFT list (Work In Progress)
Headers: ['Token ID', 'Token Name', 'Token decimals']
Row: ['1', 'ERGO', '0']
Row: ['03faf2cb329f2e90d6d23b58d91bbb6c046aa143261cc21f52fbe2824bfcbf04', 'SigUSD', '2']
Row: ['8b08cdd5449a9592a9e79711d7d79249d7a03c535d17efaee83e216e80a44c4b', 'RSN', '3']
Row: ['01dce8a5632d19799950ff90bca3b5d0ca3ebfa8aaafd06f0cc6d

In [51]:
df.to_csv('supported-tokens.csv')